# Семинар 8: Практика инжиниринга признаков и логистической регрессии

**Цели семинара:**
1.  Провести EDA для задачи классификации, выявляя категориальные признаки, скрытые в числовых колонках.
2.  Применить One-Hot Encoding для категориальных признаков.
3.  Применить `StandardScaler` для масштабирования числовых признаков.
4.  Обучить модель `LogisticRegression` с автоматическим подбором гиперпараметров через `GridSearchCV`.
5.  Научиться строить и подробно интерпретировать матрицу ошибок, отчет о классификации и ROC-кривую.

## 1. Загрузка и исследовательский анализ данных (EDA)

Мы будем работать с набором данных `heart.csv`, который содержит информацию о пациентах и наличии у них заболевания сердца.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/data/heart.csv')
df.head()

In [ ]:
df.info()

In [ ]:
sns.countplot(x='target', data=df)
plt.title('Распределение целевой переменной')
plt.show()

**Ключевой момент EDA:** Некоторые признаки, такие как `cp` (тип боли в груди), `thal` (состояние сосудов) и `slope` (наклон сегмента ST), представлены числами, но по своей сути являются **категориями**. Линейные модели будут неверно интерпретировать их как упорядоченные числовые значения. Поэтому их необходимо преобразовать.

## 2. Шаг 1: Инжиниринг признаков и подготовка данных

### 2.1. One-Hot Encoding

Мы применим One-Hot Encoding к категориальным признакам. Это создаст новые бинарные столбцы для каждой категории.

In [ ]:
X = df.drop('target', axis=1)
y = df['target']

# Указываем столбцы, которые являются категориальными
categorical_cols = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']

# Применяем pd.get_dummies
X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

print("Размерность X после OHE:", X.shape)
X.head()

### 2.2. Разделение на выборки и масштабирование

Теперь, когда все данные числовые, мы можем разделить их на обучающую и тестовую выборки, а затем применить масштабирование.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
# Обучаем scaler ТОЛЬКО на обучающих данных
scaled_X_train = scaler.fit_transform(X_train)
# Применяем обученный scaler к обеим выборкам
scaled_X_test = scaler.transform(X_test)

## 3. Шаг 2: Обучение модели

Мы используем `GridSearchCV` для автоматического поиска лучших гиперпараметров `C` (сила регуляризации) и `penalty` (тип регуляризации).

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Создаем модель
log_model = LogisticRegression(solver='liblinear')

# Создаем сетку гиперпараметров
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': np.logspace(-3, 3, 7)
}

# Создаем объект GridSearchCV
grid_search = GridSearchCV(log_model, param_grid, cv=5, scoring='accuracy')

# Обучаем
grid_search.fit(scaled_X_train, y_train)

print("Лучшие параметры:", grid_search.best_params_)

## 4. Шаг 3: Оценка модели

Теперь оценим лучшую модель, найденную GridSearchCV, на тестовых данных.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    RocCurveDisplay,
    classification_report
)

# Предсказания
y_pred = grid_search.predict(scaled_X_test)

# Матрица ошибок (замена plot_confusion_matrix)
print("Матрица ошибок:")
ConfusionMatrixDisplay.from_estimator(
    grid_search,            # fitted estimator (например, GridSearchCV)
    scaled_X_test,
    y_test
)
plt.show()

# Отчет о классификации
print("\nОтчет о классификации:")
print(classification_report(y_test, y_pred))

# ROC-кривая (замена plot_roc_curve)
# Работает, если у модели есть predict_proba или decision_function
print("\nROC-кривая:")
RocCurveDisplay.from_estimator(grid_search, scaled_X_test, y_test)
plt.show()

### 3.4. Как читать отчет по классификации: практическое руководство

После того как модель обучена, `scikit-learn` предоставляет мощные инструменты для ее анализа: `confusion_matrix` и `classification_report`. Давайте разберемся, как их правильно интерпретировать.

#### **1. Матрица ошибок (Confusion Matrix)**

Это "рентгеновский снимок" вашей модели. Он показывает, где именно модель права, а где ошибается.

|                | **Предсказано: 0** (Negative) | **Предсказано: 1** (Positive) |
|:---:|:---:|:---:|
| **Факт: 0** (Negative) | TN (True Negative) | FP (False Positive) |
| **Факт: 1** (Positive) | FN (False Negative) | TP (True Positive) |

-   **TP (True Positive) — Истинно-положительный результат:** Модель правильно предсказала класс **1**. *(Нашла больного)*.
-   **TN (True Negative) — Истинно-отрицательный результат:** Модель правильно предсказала класс **0**. *(Нашла здорового)*.
-   **FP (False Positive) — Ложно-положительный результат (Ошибка I рода):** Модель предсказала **1**, а на самом деле был **0**. *(Ложная тревога: назвала здорового больным)*.
-   **FN (False Negative) — Ложно-отрицательный результат (Ошибка II рода):** Модель предсказала **0**, а на самом деле был **1**. *(Пропуск: назвала больного здоровым)*.

**Ключевой вывод:**
*   Если для вас критически важно **избежать ложных тревог** (например, в спам-фильтре), вы должны стремиться минимизировать **FP**.
*   Если для вас критически важно **ничего не пропустить** (например, в медицинской диагностике), вы должны стремиться минимизировать **FN**.

---

#### **2. Отчет о классификации (classification_report)**

Этот отчет агрегирует информацию из матрицы ошибок в удобные для анализа метрики.

**Метрики для каждого класса:**

*   **Precision (Точность)**
    <br>
    $$ Precision = \frac{TP}{TP + FP} $$
    <br>
    Показывает, какая доля объектов, названных моделью положительными, **действительно** является таковыми. Высокая точность означает, что если модель сказала "Да", то ей можно доверять.

*   **Recall (Полнота)**
    <br>
    $$ Recall = \frac{TP}{TP + FN} $$
    <br>
    Показывает, какую долю **всех реальных** положительных объектов модель смогла найти. Высокая полнота означает, что модель хорошо обнаруживает нужный класс.

*   **F1-score**
    <br>
    $$ F1 = 2 \cdot \frac{Precision \cdot Recall}{Precision + Recall} $$
    <br>
    Гармоническое среднее между `precision` и `recall`. Это полезная сводная метрика, особенно при несбалансированных классах, так как она будет низкой, если одна из двух компонент (precision или recall) очень низкая.

*   **Support**
    Количество реальных объектов каждого класса в тестовой выборке. Всегда смотрите на это число: если `support` для какого-то класса очень мал (например, 5-10 объектов), то метрики для этого класса будут статистически ненадежными.

**Сводные строки:**

*   **Accuracy:** Общая доля правильных предсказаний по всем классам. Полезна только на сбалансированных выборках.
*   **Macro avg:** Среднее арифметическое метрик по всем классам. Дает каждому классу равный вес, независимо от его размера. Показывает, насколько хорошо модель работает "в среднем по всем классам".
*   **Weighted avg:** Средневзвешенное значение метрик с учетом `support` каждого класса. Показывает, насколько хорошо модель работает "в среднем по всем объектам".

---

#### **3. ROC-кривая и AUC**

ROC-кривая визуализирует компромисс между двумя типами ошибок при изменении порога принятия решения.

*   **TPR (True Positive Rate / Recall):** Доля верно найденных положительных объектов. $$ TPR = \frac{TP}{TP + FN} $$
*   **FPR (False Positive Rate):** Доля ложных срабатываний. $$ FPR = \frac{FP}{FP + TN} $$

**AUC — площадь под ROC-кривой (Area Under the Curve):**
Это единое число, которое оценивает общую способность модели разделять классы.

*   **0.9 - 1.0:** Отличная модель.
*   **0.8 - 0.9:** Очень хорошая модель.
*   **0.7 - 0.8:** Хорошая модель.
*   **0.6 - 0.7:** Удовлетворительная модель.
*   **0.5 - 0.6:** Плохая модель.
*   **0.5:** Случайное угадывание (бесполезная модель).

---

#### **4. Выбор порога (Threshold)**

По умолчанию, модель относит объект к классу 1, если его вероятность > `0.5`. Этот порог можно изменять, если цена ошибок FP и FN для вас разная.

*   **Низкий порог (например, 0.3):** Увеличивает **recall** (находим больше положительных), но снижает **precision** (больше ложных тревог).
*   **Высокий порог (например, 0.7):** Увеличивает **precision** (предсказания очень надежны), но снижает **recall** (пропускаем больше положительных).

---

#### **5. Чек-лист анализа модели классификации**

1.  Посмотри `support`: Сбалансированы ли классы в тестовой выборке?
2.  Посмотри **матрицу ошибок**: Где модель ошибается чаще всего — **FP** или **FN**? Какая ошибка для вашей бизнес-задачи "больнее"?
3.  Сравни **`precision`** и **`recall`** для интересующего класса: Есть ли между ними сильный перекос?
4.  Сравни **`macro avg`** и **`weighted avg`**: Если они сильно отличаются, значит, модель плохо справляется с редким классом.
5.  Посмотри **`AUC`**: Насколько хорошо модель в принципе способна разделять классы?
6.  При необходимости (если цена ошибок разная) — подбери оптимальный **порог**, чтобы сбалансировать `precision` и `recall` под вашу задачу.

*   **AUC ≈ 0.93**

---

#### **Анализ по чек-листу:**

1.  **`support` (баланс классов):**
    *   Класс 0: 29 объектов.
    *   Класс 1: 32 объекта.
    *   **Вывод:** Классы практически идеально сбалансированы. Это значит, что метрика `accuracy` является надежной и не вводит нас в заблуждение.

2.  **`precision` и `recall` по классам:**
    *   **Класс 0:** `precision` = 0.86, `recall` = 0.86. Модель одинаково хорошо избегает ложных срабатываний и пропусков для этого класса.
    *   **Класс 1:** `precision` = 0.88, `recall` = 0.88. Модель также демонстрирует сбалансированную точность и полноту для второго класса.
    *   **Вывод:** Сильного перекоса между `precision` и `recall` нет ни для одного из классов, что говорит о стабильной работе модели.

3.  **Сравнение `macro avg` и `weighted avg`:**
    *   `macro avg f1-score` = 0.87.
    *   `weighted avg f1-score` = 0.87.
    *   **Вывод:** Значения практически идентичны, что еще раз подтверждает вывод о хорошем балансе классов. Модель не "жертвует" качеством на одном классе ради другого.

4.  **`AUC` (качество разделения):**
    *   **AUC ≈ 0.93**.
    *   **Вывод:** Это очень высокое значение. Оно означает, что модель **отлично разделяет** объекты двух классов. С высокой вероятностью она присвоит случайно выбранному объекту класса 1 более высокую оценку (вероятность), чем случайно выбранному объекту класса 0.

---

#### **Краткое резюме по модели**

*   **Стабильность:** Модель работает одинаково хорошо на обоих классах, без явных перекосов.
*   **Разделяющая способность:** Модель уверенно отличает один класс от другого (высокий `AUC`).
*   **Общее качество:** Высокое (Accuracy ≈ 87%, F1 ≈ 87%).

**Следующий шаг:** По умолчанию, модель работает с порогом `0.5`, что дает хороший баланс `precision` и `recall`. Если для бизнес-задачи цена ошибки **False Negative** (пропустить больного) гораздо выше, чем цена **False Positive** (ложная тревога), то следующим шагом мог бы стать **подбор более низкого порога** (например, `0.4`) для увеличения `recall`.